In [2]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
import math
from sklearn.linear_model import Lasso as Lasso_Reg
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score as val_score
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split as sk_split
from statsmodels.regression.linear_model import OLS
%matplotlib inline

In [3]:
# read in disease rates
column_headers = ["Country", "Year", "Mortality", "Mortality 2", "Mortality 3"]
diabetes = pd.read_csv('datasets/diabetes.csv', names = column_headers)
cardio = pd.read_csv('datasets/cardiovascular.csv', names = column_headers)
cancer= pd.read_csv('datasets/Cancer.csv', names = column_headers)

In [4]:
cardio.iloc[3:, 0:3].head(n=5)
#cardio.columns = ['Country', 'Year', 'Mortality']

,Country,Year,Mortality
3,Afghanistan,2012,511.5
4,Afghanistan,2000,542.3
5,Albania,2012,436.2
6,Albania,2000,560.5
7,Algeria,2012,396.6


In [5]:
cardio_2012 = cardio[(cardio["Year"] == "2012")] 
cardio_2000 = cardio[(cardio["Year"] == "2000")] 

In [6]:
diabetes_2012 = diabetes[(diabetes['Year'] == '2012')] 
diabetes_2000 = diabetes[(diabetes['Year'] == '2000')] 
diabetes_2000

,Country,Year,Mortality,Mortality 2,Mortality 3
4,Afghanistan,2000,34.0,35.1,32.8
6,Albania,2000,8.3,8.2,8.3
8,Algeria,2000,67.2,62.1,73.4
10,Angola,2000,35.0,36.4,33.2
12,Argentina,2000,19.9,16.0,25.2
14,Armenia,2000,32.2,33.6,29.2
16,Australia,2000,10.2,8.0,13.0
18,Austria,2000,15.5,12.4,19.3
20,Azerbaijan,2000,19.6,21.4,16.6
22,Bahamas,2000,45.6,41.5,49.9


In [7]:
cancer_2012 = cancer[(cancer['Year'] == ' 2012')] 
cancer_2000 = cancer[(cancer['Year'] == ' 2000')] 
cancer_2012

,Country,Year,Mortality,Mortality 2,Mortality 3
3,Afghanistan,2012,123.6,110.6,142.2
5,Albania,2012,123.1,105.3,142.5
7,Algeria,2012,80.6,78.0,83.9
9,Angola,2012,89.6,86.1,97.9
11,Argentina,2012,131.5,109.1,166.5
13,Armenia,2012,219.6,175.3,277.1
15,Australia,2012,111.0,90.6,135.9
17,Austria,2012,122.0,99.7,152.3
19,Azerbaijan,2012,112.8,88.5,145.0
21,Bahamas,2012,103.8,88.1,131.8


In [8]:
cardio_2012.head(n=5)

,Country,Year,Mortality,Mortality 2,Mortality 3
3,Afghanistan,2012,511.5,520.9,498.0
5,Albania,2012,436.2,417.6,447.4
7,Algeria,2012,396.6,359.9,434.6
9,Angola,2012,374.6,369.8,376.4
11,Argentina,2012,191.7,148.4,247.8


In [9]:
cardio_2000.head(n=5)

,Country,Year,Mortality,Mortality 2,Mortality 3
4,Afghanistan,2000,542.3,554.1,523.1
6,Albania,2000,560.5,494.7,638.6
8,Algeria,2000,434.7,401.2,472.2
10,Angola,2000,383.4,385.2,375.2
12,Argentina,2000,226.6,169.1,302.8


In [15]:
cardio_lands = cardio_2000['Country'].unique()
diabetes_lands = diabetes_2000['Country'].unique()
cancer_lands = cancer_2000['Country'].unique()

In [16]:
df = pd.read_csv('datasets/allwregions_92_00.csv')
df.head(n=5)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1992,1992,kg,1.08,Fc,Calculated data
1,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1993,1993,kg,1.00,Fc,Calculated data
2,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1994,1994,kg,0.87,Fc,Calculated data
3,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1995,1995,kg,0.69,Fc,Calculated data
4,CC,Food Supply - Crops Primary Equivalent,2,Afghanistan,645,Food supply quantity (kg/capita/yr),2617,Apples and products,1996,1996,kg,0.87,Fc,Calculated data


In [18]:
countries_all = df['Country'].unique()
predictor_countries = []
for country in countries_all:
    if country in cancer_lands:
        predictor_countries.append(country)

In [44]:
predictor_countries = ['Canada',
 'Turkmenistan',
 'Lithuania',
 'Cambodia',
 'Ethiopia',
 'Swaziland',
 'Argentina',
 'Cameroon',
 'Burkina Faso',
 'Ghana',
 'Saudi Arabia',
 'Japan',
 'Slovenia',
 'Guatemala',
 'Bosnia and Herzegovina',
 'Guinea',
 'Russian Federation',
 'Germany',
 'Spain',
 'Liberia',
 'Maldives',
 'Paraguay',
 'Pakistan',
 'Oman',
 'Cabo Verde',
 'Gabon',
 'New Zealand',
 'Yemen',
 'Jamaica',
 'Albania',
 'United Arab Emirates',
 'Viet Nam',
 'Azerbaijan',
 'Lesotho',
 'Kenya',
 'Tajikistan',
 'Turkey',
 'Afghanistan',
 'Bangladesh',
 'Iran (Islamic Republic of)',
 'India',
 'Mongolia',
 'France',
 'Rwanda',
 'Slovakia',
 'Peru',
 'Norway',
 'Malawi',
 'Benin',
 'Cuba',
 'United States of America',
 'Togo',
 'China',
 'Armenia',
 'Republic of Korea',
 'Dominican Republic',
 'Bolivia (Plurinational State of)',
 'Ukraine',
 'Finland',
 'Indonesia',
 'Central African Republic',
 'Mauritius',
 'Sweden',
 'Australia',
 'Mali',
 'Bulgaria',
 'Romania',
 'Angola',
 'Chad',
 'South Africa',
 'Fiji',
 'Brunei Darussalam',
 'Malaysia',
 'Austria',
 'Mozambique',
 'Uganda',
 'Hungary',
 'Niger',
 'Brazil',
 'Kuwait',
 'Panama',
 'Republic of Moldova',
 'Costa Rica',
 'Luxembourg',
 'Bahamas',
 'Ireland',
 'Nigeria',
 'Ecuador',
 'Belarus',
 'Algeria',
 'El Salvador',
 'Solomon Islands',
 'Chile',
 'Belgium',
 'Thailand',
 'Haiti',
 'Belize',
 'Sierra Leone',
 'Georgia',
 "Lao People's Democratic Republic",
 'Gambia',
 'Philippines',
 'Morocco',
 'Croatia',
 'Guinea-Bissau',
 'United Republic of Tanzania',
 'Switzerland',
 'Venezuela (Bolivarian Republic of)',
 'Iraq',
 'Portugal',
 'Estonia',
 'Uruguay',
 'Lebanon',
 'Uzbekistan',
 'Egypt',
 'Djibouti',
 'Timor-Leste',
 'Colombia',
 'Cyprus',
 'Barbados',
 'Madagascar',
 'Italy',
 'Nepal',
 'Malta',
 'Netherlands',
 'Suriname',
 'Israel',
 'Iceland',
 'Zambia',
 'Senegal',
 'Zimbabwe',
 'Jordan',
 'Denmark',
 'Kazakhstan',
 'Poland',
 'Mauritania',
 'Kyrgyzstan',
 'Trinidad and Tobago',
 'Latvia',
 'Guyana',
 'Honduras',
 'Myanmar',
 'Mexico',
 'Tunisia',
 'Nicaragua',
 "Democratic People's Republic of Korea",
 'Congo',
 'Greece',
 'Sri Lanka',
 'Namibia',
 'Botswana']

In [45]:
x_df = pd.read_csv('datasets/predictors_filled.csv')
x_df = x_df.iloc[:, 1:]
x_df.head(n=5)

,Ricebran Oil,Plantains,"Sugar, Raw Equivalent","Beverages, Alcoholic",Roots & Tuber Dry Equiv,Olives (including preserved),Cloves,"Fats, Animals, Raw",Millet and products,Coconuts - Incl Copra,...,Tomatoes and products,"Oilcrops Oil, Other",Pelagic Fish,"Butter, Ghee",Bananas,"Spices, Other","Meat, Aquatic Mammals",Sweet potatoes,"Oranges, Mandarines",Rice (Milled Equivalent)
0,0.00,0.603333,47.554444,1.558889,16.850000,0.532222,0.002222,6.007778,0.000000,1.112222,...,30.738889,0.102222,5.432222,2.918889,13.284444,0.297778,0.0,0.466667,47.530000,6.638889
1,0.00,0.000000,14.420000,0.327778,3.318889,0.001111,0.000000,0.953333,0.826667,0.000000,...,31.094444,0.203333,0.008889,1.820000,0.043333,0.004444,0.0,0.000000,1.604444,7.101111
2,0.00,0.000000,30.954444,5.077778,24.973333,0.024444,0.000000,3.285556,0.000000,0.254444,...,4.458889,2.487778,12.195556,3.835556,4.492222,0.100000,0.0,0.000000,4.731111,1.540000
3,0.03,0.583333,4.422222,1.601111,3.670000,0.000000,0.011111,0.813333,0.000000,0.766667,...,0.000000,0.098889,0.363333,0.002222,11.788889,0.006667,0.0,3.337778,4.556667,167.165556
4,0.00,26.812222,4.042222,0.096667,17.707778,0.000000,0.001111,0.238750,3.291111,0.167778,...,0.943333,0.405556,0.000000,0.198889,2.028889,0.051111,0.0,4.751111,0.402222,1.253333


In [46]:
cancer = []
for country in predictor_countries:
    for i, place in enumerate(cancer_2000['Country'].values):
        if place == country:
            cancer.append(cancer_2000['Mortality'].values[i])
            
cancer_df = pd.DataFrame(cancer, columns = ['Cancer Mortality Rate'])
print cancer_df.shape
cancer_df.head(n=5)
            

(151, 1)


,Cancer Mortality Rate
0,142.1
1,104.9
2,154.6
3,110.9
4,100.7


In [47]:
cancerreg = LinReg()
cancerreg.fit(x_df, cancer_df)
print cancerreg.score(x_df, cancer_df)
cancerreg.coef_
cancer_coefs = cancerreg.coef_[0]
cancerreg.coef_
preds = cancerreg.predict(x_df)
#print preds
predslist = []
for pred in preds:
    predslist.append(pred[0])
predslist

0.854272257338


[138.89365665767187,
 114.99959264670238,
 146.99173265866958,
 110.56482020696663,
 96.466115821174327,
 100.8390962020118,
 139.74172850160463,
 73.925830517734227,
 118.56089057384693,
 82.65376972720756,
 83.441122662820504,
 114.39710292179612,
 172.83331407924211,
 112.96505751499231,
 134.27723132498434,
 117.04461499262348,
 159.41537290435113,
 145.8523506705873,
 122.02423979461369,
 106.99890589868542,
 118.49456279481333,
 128.5734242002533,
 75.925858189079605,
 65.817040948789014,
 90.018859514874435,
 66.005010488871136,
 157.20668156057278,
 84.876867938332595,
 135.96075258015358,
 114.04594658099055,
 145.92754454328193,
 117.81767162846032,
 143.74370742049055,
 74.217521631960295,
 119.34111549794889,
 86.511713662388189,
 157.27890855395222,
 120.97800827169581,
 109.41247728629483,
 114.56900960376393,
 82.220897249858041,
 280.23368472159439,
 168.65872906121663,
 162.03502127725565,
 177.08404703606371,
 127.75765735752671,
 134.33979826890814,
 141.668210059648

In [48]:
#print predictors

In [49]:
print max(cancer_coefs)
print list(cancer_coefs).index(max(cancer_coefs))
#print x_df.columns[-1]
#print list(diabetes_coefs[:-1]).index(max(diabetes_coefs[:-1]))
#print x_df.columns[list(diabetes_coefs[:-1]).index(max(diabetes_coefs[:-1]))]

1372.09248473
59


In [50]:
country_indices = []
index = 0
for country in predictor_countries:
    country_indices.append(country)
print country_indices[:10]
print len(country_indices)

['Canada', 'Turkmenistan', 'Lithuania', 'Cambodia', 'Ethiopia', 'Swaziland', 'Argentina', 'Cameroon', 'Burkina Faso', 'Ghana']
151


In [51]:
maxpredictors = []
print len(x_df.columns)
print len(x_df.iloc[0, :].values)
for i in range(x_df.shape[0]):
    preds = x_df.iloc[i, :].values
    maxpredictor, index = 0, 0
    for j, val in enumerate(preds):
        pred = val * cancer_coefs[j]
        if pred > maxpredictor:
            maxpredictor = pred
            index = j
    maxpredictors.append((country_indices[i], x_df.columns[index]))    

print maxpredictors

103
103
[('Canada', 'Sugar, Refined Equiv'), ('Turkmenistan', 'Sugar, Refined Equiv'), ('Lithuania', 'Sugar, Refined Equiv'), ('Cambodia', 'Rice (Milled Equivalent)'), ('Ethiopia', 'Sugar, Refined Equiv'), ('Swaziland', 'Sugar, Refined Equiv'), ('Argentina', 'Sugar, Refined Equiv'), ('Cameroon', 'Sugar, Refined Equiv'), ('Burkina Faso', 'Rice (Milled Equivalent)'), ('Ghana', 'Rice (Milled Equivalent)'), ('Saudi Arabia', 'Sugar, Refined Equiv'), ('Japan', 'Rice (Milled Equivalent)'), ('Slovenia', 'Sugar, Refined Equiv'), ('Guatemala', 'Sugar, Refined Equiv'), ('Bosnia and Herzegovina', 'Sugar, Refined Equiv'), ('Guinea', 'Rice (Milled Equivalent)'), ('Russian Federation', 'Sugar, Refined Equiv'), ('Germany', 'Sugar, Refined Equiv'), ('Spain', 'Sugar, Refined Equiv'), ('Liberia', 'Rice (Milled Equivalent)'), ('Maldives', 'Sugar, Refined Equiv'), ('Paraguay', 'Sugar, Refined Equiv'), ('Pakistan', 'Sugar, Refined Equiv'), ('Oman', 'Rice (Milled Equivalent)'), ('Cabo Verde', 'Rice (Milled E

In [52]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
cancerreg = Lasso(alpha = .00001, normalize=True)
cancerreg.fit(x_df, cancer_df)
print cancerreg.score(x_df, cancer_df)
coefficients = cancerreg.coef_
print 'hi'
print len(coefficients)
print len([i for i, item in enumerate(coefficients) if abs(item) >0])

cancerreg = LassoCV(alphas = [.01, .1], normalize=True)
#cancerlist = list(cancer_df)
cancerreg.fit(x_df, cancer_df.values.ravel())
print cancerreg.score(x_df, cancer_df)

0.809983319291
hi
103
101
0.59362415938


In [53]:
from sklearn.linear_model import Ridge
cancerreg = Ridge(alpha = .0001, normalize=True)
cancerreg.fit(x_df, cancer_df)
cancerreg.score(x_df, cancer_df)

0.81099608209243479

In [54]:
diabetes = []
for country in predictor_countries:
    for i, place in enumerate(diabetes_2000['Country'].values):
        if place == country:
            diabetes.append(diabetes_2000['Mortality'].values[i])
            
diabetes_df = pd.DataFrame(diabetes, columns = ['Diabetes Mortality Rate'])
print diabetes_df.shape
diabetes_df.head(n=5)
            

(151, 1)


,Diabetes Mortality Rate
0,14.0
1,19.8
2,5.0
3,15.9
4,28.9


In [55]:
diabetes_reg = LinReg()
diabetes_reg.fit(x_df, diabetes_df)
print diabetes_reg.score(x_df, diabetes_df)
print diabetes_reg.coef_
diabetes_coefs = diabetes_reg.coef_[0]
print max(diabetes_coefs)
print list(diabetes_coefs).index(max(diabetes_coefs))
print x_df.columns[-1]
print list(diabetes_coefs[:-1]).index(max(diabetes_coefs[:-1]))
print x_df.columns[list(diabetes_coefs[:-1]).index(max(diabetes_coefs[:-1]))]
for i, coef in enumerate(diabetes_coefs):
    if coef < -2000:
        print i
x_df.columns[48]

0.81543119032
[[ -7.02696211e+01  -1.89098499e-02   3.51409933e-01   7.51998156e-01
    4.84780528e-01  -3.45649606e+00  -2.39383606e+01   9.12715956e-01
    1.91645886e-02   6.59865501e-01  -1.48592142e+00   7.84487272e-02
    3.36302905e+00  -1.18711903e+00   6.93499334e-03  -2.03500648e-01
   -1.36348809e+00  -8.69710925e-01  -1.03583414e+01   1.68364005e-01
   -1.15301124e+00  -2.53938573e+00   6.70144593e+00   5.61211562e-01
    1.80165039e+01   8.26226919e-01   1.67407598e-01   1.90991358e-01
    5.36470560e+00  -1.20589620e+00  -3.95384590e-01   7.23832005e-01
    4.28256959e+02  -1.80553649e-01   3.98036017e-01  -2.83073973e+00
   -4.38131691e-02   2.31149429e+00  -2.07790769e-01  -3.27268819e+00
    6.04818433e+00   1.83568843e-01   2.25012925e-01  -2.91362555e+01
    7.71341840e-02   2.71036354e-01   3.82905001e+00   1.26810795e+00
    2.15140235e+02   1.34401427e-01  -1.74047495e-01  -1.09499474e+00
   -2.46721460e+00  -2.03837554e+00   1.02765192e+00   2.62894480e+00
   -1.

'Rice (Paddy Equivalent)'

In [56]:
preds = diabetes_reg.predict(x_df)
#print preds
predslist = []
for pred in preds:
    predslist.append(pred[0])
predslist

[7.0435398600552404,
 32.346149214972662,
 14.0660366490524,
 15.883538138684322,
 33.335166798699483,
 83.817242043571667,
 18.995788035903139,
 32.391840379177644,
 55.572707676844381,
 32.292524635662609,
 41.717694982057743,
 7.534088231605871,
 29.265415642823179,
 50.866005598794551,
 22.125469673556786,
 35.766772415640823,
 4.7604626063830224,
 10.28114474390302,
 14.239007889623259,
 44.252029784557614,
 28.443092585878709,
 40.954777003070355,
 32.99227247219094,
 58.86480471490993,
 33.350561030641053,
 25.305479506549979,
 18.297105578640618,
 35.51199483743401,
 72.25970550600411,
 16.754421338601766,
 39.5897144645214,
 23.739476845646074,
 31.224141781743334,
 79.073147589142508,
 25.204838866983302,
 21.587276384387884,
 23.990629793216591,
 29.764372826839441,
 42.624145828903217,
 8.2459527794462488,
 30.612528525943141,
 9.2080380254499019,
 11.454289844323263,
 29.130071628537259,
 9.121977773356754,
 30.310154865920573,
 28.735473620462905,
 23.847525297778681,
 35

In [57]:
diabetes_reg.coef_

array([[ -7.02696211e+01,  -1.89098499e-02,   3.51409933e-01,
          7.51998156e-01,   4.84780528e-01,  -3.45649606e+00,
         -2.39383606e+01,   9.12715956e-01,   1.91645886e-02,
          6.59865501e-01,  -1.48592142e+00,   7.84487272e-02,
          3.36302905e+00,  -1.18711903e+00,   6.93499334e-03,
         -2.03500648e-01,  -1.36348809e+00,  -8.69710925e-01,
         -1.03583414e+01,   1.68364005e-01,  -1.15301124e+00,
         -2.53938573e+00,   6.70144593e+00,   5.61211562e-01,
          1.80165039e+01,   8.26226919e-01,   1.67407598e-01,
          1.90991358e-01,   5.36470560e+00,  -1.20589620e+00,
         -3.95384590e-01,   7.23832005e-01,   4.28256959e+02,
         -1.80553649e-01,   3.98036017e-01,  -2.83073973e+00,
         -4.38131691e-02,   2.31149429e+00,  -2.07790769e-01,
         -3.27268819e+00,   6.04818433e+00,   1.83568843e-01,
          2.25012925e-01,  -2.91362555e+01,   7.71341840e-02,
          2.71036354e-01,   3.82905001e+00,   1.26810795e+00,
        

In [58]:
maxpredictors = []
print len(x_df.columns)
print len(x_df.iloc[0, :].values)
for i in range(x_df.shape[0]):
    preds = x_df.iloc[i, :].values
    maxpredictor, index = 0, 0
    for j, val in enumerate(preds):
        pred = val * diabetes_coefs[j]
        if pred > maxpredictor:
            maxpredictor = pred
            index = j
    maxpredictors.append((country_indices[i], x_df.columns[index]))    

for country, crop in maxpredictors:
    if crop != 'Sugar, Refined Equiv' and crop != 'Rice (Milled Equivalent)':
        print country, crop

103
103
Canada Sugar (Raw Equivalent)
Turkmenistan Sugar (Raw Equivalent)
Lithuania Sugar (Raw Equivalent)
Cambodia Rice (Paddy Equivalent)
Ethiopia Sugar (Raw Equivalent)
Swaziland Sugar (Raw Equivalent)
Argentina Sugar (Raw Equivalent)
Cameroon Rice (Paddy Equivalent)
Burkina Faso Rice (Paddy Equivalent)
Ghana Rice (Paddy Equivalent)
Saudi Arabia Sugar (Raw Equivalent)
Japan Rice (Paddy Equivalent)
Slovenia Sugar (Raw Equivalent)
Guatemala Sugar (Raw Equivalent)
Bosnia and Herzegovina Sugar (Raw Equivalent)
Guinea Rice (Paddy Equivalent)
Russian Federation Sugar (Raw Equivalent)
Germany Sugar (Raw Equivalent)
Spain Sugar (Raw Equivalent)
Liberia Rice (Paddy Equivalent)
Maldives Rice (Paddy Equivalent)
Paraguay Sugar (Raw Equivalent)
Pakistan Sugar (Raw Equivalent)
Oman Rice (Paddy Equivalent)
Cabo Verde Rice (Paddy Equivalent)
Gabon Rice (Paddy Equivalent)
New Zealand Sugar (Raw Equivalent)
Yemen Sugar (Raw Equivalent)
Jamaica Sugar (Raw Equivalent)
Albania Sugar (Raw Equivalent)
Uni

In [59]:
diabetes_reg = Lasso(alpha=.001, normalize=True)
diabetes_reg.fit(x_df, diabetes_df)
print diabetes_reg.score(x_df, diabetes_df)
diabetes_reg.coef_
for col in x_df.columns:
    print '"' + col + '"' + ', '

0.809505996698
"Ricebran Oil", 
"Plantains", 
"Sugar, Raw Equivalent", 
"Beverages, Alcoholic", 
"Roots & Tuber Dry Equiv", 
"Olives (including preserved)", 
"Cloves", 
"Fats, Animals, Raw", 
"Millet and products", 
"Coconuts - Incl Copra", 
"Eggs", 
"Vegetables, Other", 
"Sesame seed", 
"Marine Fish, Other", 
"Wine", 
"Pigmeat", 
"Apples and products", 
"Rape and Mustard Oil", 
"Fish, Liver Oil", 
"Maize and products", 
"Sweeteners, Other", 
"Fish, Body Oil", 
"Honey", 
"Demersal Fish", 
"Groundnuts (Shelled Eq)", 
"Barley and products", 
"Maize Germ Oil", 
"Beer", 
"Groundnut Oil", 
"Sugar non-centrifugal", 
"Pineapples and products", 
"Pulses, Other and products", 
"Sugar (Raw Equivalent)", 
"Meat, Other", 
"Molasses", 
"Pimento", 
"Palm Oil", 
"Oilcrops, Other", 
"Dates", 
"Oats", 
"Soyabeans", 
"Freshwater Fish", 
"Beans", 
"Sugar beet", 
"Cephalopods", 
"Mutton & Goat Meat", 
"Sesameseed Oil", 
"Grapes and products (excl wine)", 
"Rice (Paddy Equivalent)", 
"Beverages, Fermented"

In [60]:
cardio = []
for country in predictor_countries:
    for i, place in enumerate(cardio_2000['Country'].values):
        if place == country:
            cardio.append(cardio_2000['Mortality'].values[i])
            
cardio_df = pd.DataFrame(cardio, columns = ['Cardio Mortality Rate'])
print cardio_df.shape
cardio_df.head(n=5)
            

(151, 1)


,Cardio Mortality Rate
0,150.8
1,724.4
2,373.9
3,222.5
4,180.4


In [146]:
cardio_reg = LinReg()
cardio_reg.fit(x_df, cardio_df)
print cardio_reg.score(x_df, cardio_df)
print cross_val_score(cardio_reg, x_df.iloc[:, :-1], cardio_df)
cardio_coefs = cardio_reg.coef_[0]
cardio_coefs

1.0
[-310.27781966 -344.95282988 -396.53174395]


array([  5.94507886e-14,  -3.74700271e-16,  -2.04003481e-15,
         2.60902411e-15,   0.00000000e+00,   1.16018306e-14,
         3.43943624e-14,  -4.99600361e-16,   1.04083409e-15,
        -1.33226763e-15,  -1.04083409e-15,   2.63677968e-16,
         3.21465944e-16,   1.60028241e-15,   4.51028104e-17,
         3.78169718e-16,   9.66240976e-16,   2.08123449e-15,
        -1.06804045e-13,   1.45716772e-16,   5.30825384e-16,
         3.33034381e-15,  -1.17715625e-14,  -2.49453236e-15,
        -2.23475752e-14,   1.04083409e-17,   5.48595457e-15,
         5.68989300e-16,  -8.17773041e-15,   3.41957365e-15,
        -1.07639592e-15,  -1.43548368e-15,  -1.08535750e-12,
        -9.94755493e-16,  -9.40378553e-14,   7.19910243e-17,
         4.44956572e-15,  -6.10947924e-16,   9.68409380e-16,
         3.30378086e-15,  -8.53950157e-15,   1.96370697e-15,
         2.79680792e-15,   2.36179823e-14,   1.52646992e-14,
         3.73832909e-16,   6.42378945e-15,  -1.87263399e-15,
        -6.49660881e-13,

In [62]:
preds = cardio_reg.predict(x_df)
#print preds
predslist = []
for pred in preds:
    predslist.append(pred[0])
predslist

[137.66503030416163,
 656.20611797564663,
 431.88231217760779,
 171.06478210433954,
 191.13747167059967,
 385.41744512382968,
 181.46567340537251,
 220.41498447595325,
 229.18925300259536,
 351.88318822418034,
 399.56944159075022,
 133.05139210797017,
 255.01809350511496,
 212.67155424684563,
 352.6756009679417,
 285.14099331031423,
 519.46348571193732,
 191.97568583778241,
 172.38210269445801,
 283.98274648981271,
 349.9898167116927,
 238.94103296336937,
 301.90864433004856,
 381.16713629441904,
 369.25934472654001,
 296.05931646404599,
 215.83200999090434,
 370.85094157834931,
 244.55126197778839,
 512.9353995444518,
 357.6223263780131,
 308.92819006688762,
 550.84329764554889,
 385.7496939094591,
 238.5036487940007,
 571.91615161414404,
 508.60660059040777,
 501.82280468422789,
 258.23800177453188,
 481.70554856315823,
 325.2190818259852,
 681.35516314254812,
 181.90109049656996,
 266.61990793642974,
 400.41786538688496,
 202.34536819407469,
 253.58734788466165,
 274.20001866921405,

In [63]:
maxpredictors = []
print len(x_df.columns)
print len(x_df.iloc[0, :].values)
for i in range(x_df.shape[0]):
    preds = x_df.iloc[i, :].values
    maxpredictor, index = 0, 0
    for j, val in enumerate(preds):
        pred = val * cardio_coefs[j]
        if pred > maxpredictor:
            maxpredictor = pred
            index = j
    maxpredictors.append((country_indices[i], x_df.columns[index]))    

print maxpredictors
#for country, crop in maxpredictors:
    #if crop != 'Sugar, Refined Equiv' and crop != 'Rice (Milled Equivalent)':
        #print country, crop

103
103
[('Canada', 'Sugar (Raw Equivalent)'), ('Turkmenistan', 'Sugar (Raw Equivalent)'), ('Lithuania', 'Sugar (Raw Equivalent)'), ('Cambodia', 'Rice (Paddy Equivalent)'), ('Ethiopia', 'Sugar (Raw Equivalent)'), ('Swaziland', 'Sugar (Raw Equivalent)'), ('Argentina', 'Sugar (Raw Equivalent)'), ('Cameroon', 'Rice (Paddy Equivalent)'), ('Burkina Faso', 'Rice (Paddy Equivalent)'), ('Ghana', 'Rice (Paddy Equivalent)'), ('Saudi Arabia', 'Rice (Paddy Equivalent)'), ('Japan', 'Rice (Paddy Equivalent)'), ('Slovenia', 'Sugar (Raw Equivalent)'), ('Guatemala', 'Sugar (Raw Equivalent)'), ('Bosnia and Herzegovina', 'Sugar (Raw Equivalent)'), ('Guinea', 'Rice (Paddy Equivalent)'), ('Russian Federation', 'Sugar (Raw Equivalent)'), ('Germany', 'Sugar (Raw Equivalent)'), ('Spain', 'Sugar (Raw Equivalent)'), ('Liberia', 'Rice (Paddy Equivalent)'), ('Maldives', 'Rice (Paddy Equivalent)'), ('Paraguay', 'Sugar (Raw Equivalent)'), ('Pakistan', 'Sugar (Raw Equivalent)'), ('Oman', 'Rice (Paddy Equivalent)'), 

In [64]:

cardio_reg = Lasso(alpha=.00001, normalize=True)
cardio_reg.fit(x_df, cardio_df)
cardio_reg.score(x_df, cardio_df)

0.84503112578167394

In [ ]:
### Best Subset Selection
min_bic = 1e10 # set some initial large value for min BIC score
best_subset = [] # best subset of predictors

# Create all possible subsets of the set of 10 predictors
predictor_set = set(range(10)) # predictor set = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

# Repeat for every possible size of subset
for size_k in range(10): 
    # Create all possible subsets of size 'size', 
    # using the 'combination' function from the 'itertools' library
    subsets_of_size_k = it.combinations(predictor_set, size_k + 1) 
    
    max_r_squared = -1e10 # set some initial small value for max R^2 score
    best_k_subset = [] # best subset of predictors of size k
    
    # Iterate over all subsets of our predictor set
    for predictor_subset in subsets_of_size_k:
        
        x_subset = x_df[x_df.columns[temp]]
        
        # Fit and evaluate R^2
        y_temp = np.asarray(cardio_df[cardio_df.columns[0]])
        x = np.asarray(x_subset)
        y = []
        for val in y_temp:
            y.append(float(val))
        # Use only a subset of predictors in the training data
        x_subset = x[:, predictor_subset]

        # Fit and evaluate R^2
        model = OLS(y, x_subset)
        results = model.fit()
        r_squared = results.rsquared
        
        # Update max R^2 and best predictor subset of size k
        # If current predictor subset has a higher R^2 score than that of the best subset 
        # we've found so far, remember the current predictor subset as the best!
        if(r_squared > max_r_squared): 
            max_r_squared = r_squared
            best_k_subset = predictor_subset[:]
                

    # Use only the best subset of size k for the predictors
    x_subset = x[:, best_k_subset]
        
    # Fit and evaluate BIC of the best subset of size k
    model = OLS(y, x_subset)
    results = model.fit()
    bic = results.bic
    
    # Update minimum BIC and best predictor subset
    # If current predictor has a lower BIC score than that of the best subset 
    # we've found so far, remember the current predictor as the best!
    if(bic < min_bic): 
        min_bic = bic
        best_subset = best_k_subset[:]
    
print('Best subset by exhaustive search:')
print sorted(best_subset)

In [251]:
### Step-wise Forward Selection
d = x_df.shape[1] # total no. of predictors

# Keep track of current set of chosen predictors, and the remaining set of predictors
current_predictors = [] 
remaining_predictors = range(d)

# Set some initial large value for min BIC score for all possible subsets
global_min_bic = 1e10 

# Keep track of the best subset of predictors
best_subset = [] 

# Iterate over all possible subset sizes, 0 predictors to d predictors
for size in range(d):    
    max_r_squared = -1e10 # set some initial small value for max R^2
    best_predictor = -1 # set some throwaway initial number for the best predictor to add
    bic_with_best_predictor = 1e10 # set some initial large value for BIC score   
        
    # Iterate over all remaining predictors to find best predictor to add
    for i in remaining_predictors:
        # Make copy of current set of predictors
        temp = current_predictors[:]
        # Add predictor 'i'
        temp.append(i)
                                    
        # Use only a subset of predictors in the training data
        x_subset = x_df[x_df.columns[temp]]
        
        # Fit and evaluate R^2
        y_temp = np.asarray(cancer_df[cancer_df.columns[0]])
        x = np.asarray(x_subset)
        y = []
        for val in y_temp:
            y.append(float(val))

        model = OLS(y, x)
        #model = OLS(cardio_df, x_subset)
        results = model.fit()
        r_squared = results.rsquared
        
        # Check if we get a higher R^2 value than than current max R^2, if so, update
        if(r_squared > max_r_squared):
            max_r_squared = r_squared
            best_predictor = i
            bic_with_best_predictor = results.bic
    
    # Remove best predictor from remaining list, and add best predictor to current list
    remaining_predictors.remove(best_predictor)
    current_predictors.append(best_predictor)
    
    # Check if BIC for with the predictor we just added is lower than 
    # the global minimum across all subset of predictors
    if(bic_with_best_predictor < global_min_bic):
        best_subset = current_predictors[:]
        global_min_bic = bic_with_best_predictor
    
print 'Step-wise forward subset selection:'
print sorted(best_subset) # add 1 as indices start from 0

<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'float'>
<type 'flo

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
### Step-wise Forward Selection
d = x_df.shape[1] # total no. of predictors

# Keep track of current set of chosen predictors, and the remaining set of predictors
current_predictors = [] 
remaining_predictors = range(d)

# Set some initial large value for min BIC score for all possible subsets
global_min_bic = 1e10 
global_r2 = 0

# Keep track of the best subset of predictors
best_subset = [] 

# Iterate over all possible subset sizes, 0 predictors to d predictors
for size in range(d):    
    max_r_squared = -1e10 # set some initial small value for max R^2
    best_predictor = -1 # set some throwaway initial number for the best predictor to add
    #bic_with_best_predictor = 1e10 # set some initial large value for BIC score   
        
    # Iterate over all remaining predictors to find best predictor to add
    for i in remaining_predictors:
        # Make copy of current set of predictors
        temp = current_predictors[:]
        # Add predictor 'i'
        temp.append(i)
                                    
        # Use only a subset of predictors in the training data
        x_subset = x_df[x_df.columns[temp]]
        
        # Fit and evaluate R^2
        y_temp = np.asarray(cardio_df[cardio_df.columns[0]])
        x = np.asarray(x_subset)
        y = []
        for val in y_temp:
            y.append(float(val))
        
        model = LinReg()
        model.fit(x, y)
        r_squared = model.score(x, y)
        #model = OLS(cardio_df, x_subset)
        #results = model.fit()
        #r_squared = results.rsquared
        
        # Check if we get a higher R^2 value than than current max R^2, if so, update
        if(r_squared > max_r_squared):
            max_r_squared = r_squared
            best_predictor = i
            bic_with_best_predictor = results.bic
    
    # Remove best predictor from remaining list, and add best predictor to current list
    remaining_predictors.remove(best_predictor)
    current_predictors.append(best_predictor)
    
    # Check if BIC for with the predictor we just added is lower than 
    # the global minimum across all subset of predictors
    if(bic_with_best_predictor < global_min_bic):
        best_subset = current_predictors[:]
        global_min_bic = bic_with_best_predictor
    
print 'Step-wise forward subset selection:'
print sorted(best_subset) # add 1 as indices start from 0

In [66]:
stepwise = []
for val in best_subset:
    stepwise.append(x_df.columns[val])


In [82]:
all_df = pd.concat([x_df, cardio_df])
print len(x_df.columns)
print len(all_df.columns)
#print all_df.columns['Cardio Mortality Rate']
for vol in all_df.columns:
    if vol not in x_df.columns:
        print vol
x_df['Cardio Mortality Rate'] = cardio_df['Cardio Mortality Rate'].values
x_df.head(n=5)

104
104


,Ricebran Oil,Plantains,"Sugar, Raw Equivalent","Beverages, Alcoholic",Roots & Tuber Dry Equiv,Olives (including preserved),Cloves,"Fats, Animals, Raw",Millet and products,Coconuts - Incl Copra,...,"Oilcrops Oil, Other",Pelagic Fish,"Butter, Ghee",Bananas,"Spices, Other","Meat, Aquatic Mammals",Sweet potatoes,"Oranges, Mandarines",Rice (Milled Equivalent),Cardio Mortality Rate
0,0.00,0.603333,47.554444,1.558889,16.850000,0.532222,0.002222,6.007778,0.000000,1.112222,...,0.102222,5.432222,2.918889,13.284444,0.297778,0.0,0.466667,47.530000,6.638889,150.8
1,0.00,0.000000,14.420000,0.327778,3.318889,0.001111,0.000000,0.953333,0.826667,0.000000,...,0.203333,0.008889,1.820000,0.043333,0.004444,0.0,0.000000,1.604444,7.101111,724.4
2,0.00,0.000000,30.954444,5.077778,24.973333,0.024444,0.000000,3.285556,0.000000,0.254444,...,2.487778,12.195556,3.835556,4.492222,0.100000,0.0,0.000000,4.731111,1.540000,373.9
3,0.03,0.583333,4.422222,1.601111,3.670000,0.000000,0.011111,0.813333,0.000000,0.766667,...,0.098889,0.363333,0.002222,11.788889,0.006667,0.0,3.337778,4.556667,167.165556,222.5
4,0.00,26.812222,4.042222,0.096667,17.707778,0.000000,0.001111,0.238750,3.291111,0.167778,...,0.405556,0.000000,0.198889,2.028889,0.051111,0.0,4.751111,0.402222,1.253333,180.4


In [150]:
linreg = LinReg()
linreg.fit(x_df[stepwise], cardio_df)
linreg.score(x_df[stepwise], cardio_df)

0.6175403768904395

In [131]:
train, test = sk_split(x_df, train_size = .97)

In [133]:
x_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
x_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


print y_train.shape


(146,)


In [142]:
from sklearn.cross_validation import cross_val_score
lin = LinReg()
lin.fit(x_train, y_train)
print y_train.shape
print x_train.shape
#print linreg.score(x_train[stepwise], y_train)
print lin.score(x_train, y_train)
print cross_val_score(lin, x_train, y_train)

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

(146,)
(146, 103)
0.866088665747
[ -76.8941103  -121.98926638  -84.73693565]


In [248]:
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
def cross_validate_OLS(data_cv, folds):
    
    # split the data up into folds
    kf = KFold(data_cv.shape[0],n_folds=folds)

    total_r2_train = 0
    total_r2_test = 0
    total_mse = 0

    for train_index, test_index in kf:
        x = data_cv.iloc[:, :-1]
        y = data_cv.iloc[:, -1]

        # extract training and testing data
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        print x_train.shape
        
        y_temp = np.asarray(y_train)
        y = []
        for val in y_temp:
            y.append(float(val))

        # run ridge model
        model = OLS(y, np.asarray(x_train))
        results = model.fit()
        preds = model.predict(x_test)
        r2_train = accuracy_score(y_test, preds)
        r2_test = accuracy_score(y_test, preds)
        
        # calculate R^2 for the training set
        #r2_train = results.rsquared
        total_r2_train += r2_train
        
        # calculate R^2 for the test set
        #r2_test = results.rsquared
        total_r2_test += r2_test
    return float(total_r2_train) / float(folds), float(total_r2_test) / float(folds) 

In [249]:
cross_validate_OLS(x_df, 5)

(120, 103)


ValueError: shapes (120,103) and (31,103) not aligned: 103 (dim 1) != 31 (dim 0)

In [229]:
from sklearn.cross_validation import KFold
def cross_validate(data_cv, folds):
    
    # split the data up into folds
    kf = KFold(data_cv.shape[0],n_folds=folds)

    total_r2_train = 0
    total_r2_test = 0
    total_mse = 0

    for train_index, test_index in kf:
        x = data_cv[stepwise].iloc[:, :-1]
        y = data_cv[stepwise].iloc[:, -1]

        # extract training and testing data
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        print x_train.shape

        # run ridge model
        reg = LinReg()
        reg.fit(x_train, y_train)

        tot = 0
        for i in range(30):
            tot += (reg.predict(x_test)[i] - float(y_test.iloc[i]))**2
        
        sq_error = tot
        
        # calculate MSE
        #sq_error = np.sum((reg.predict(x_test) - y_test) ** 2)
        total_mse += float(sq_error) / float(x_test.shape[0])
        
        # calculate R^2 for the training set
        r2_train = reg.score(x_train, y_train)
        total_r2_train += r2_train
        
        # calculate R^2 for the test set
        r2_test = reg.score(x_test, y_test)
        total_r2_test += r2_test
    return float(total_mse) / float(folds), float(total_r2_train) / float(folds), float(total_r2_test) / float(folds) 

In [236]:
a, b, c = cross_validate(x_df, 5)
print a, b, c
print stepwise

(120, 11)
(121, 11)
(121, 11)
(121, 11)
(121, 11)
0.000792569930928 0.201885756211 -44.8122641406
['Wheat and products', 'Rice (Paddy Equivalent)', 'Roots & Tuber Dry Equiv', 'Maize and products', 'Oranges, Mandarines', 'Millet and products', 'Milk, Whole', 'Beer', 'Beverages, Alcoholic', 'Mutton & Goat Meat', 'Olive Oil', 'Fish, Liver Oil']


In [143]:
model = OLS(y, np.asarray(x_df[stepwise]))
results = model.fit()
results.rsquared

0.93720481230511743

In [125]:
x_df.columns
x_df.iloc[:, :-1].head(n=5)

,Ricebran Oil,Plantains,"Sugar, Raw Equivalent","Beverages, Alcoholic",Roots & Tuber Dry Equiv,Olives (including preserved),Cloves,"Fats, Animals, Raw",Millet and products,Coconuts - Incl Copra,...,Tomatoes and products,"Oilcrops Oil, Other",Pelagic Fish,"Butter, Ghee",Bananas,"Spices, Other","Meat, Aquatic Mammals",Sweet potatoes,"Oranges, Mandarines",Rice (Milled Equivalent)
0,0.00,0.603333,47.554444,1.558889,16.850000,0.532222,0.002222,6.007778,0.000000,1.112222,...,30.738889,0.102222,5.432222,2.918889,13.284444,0.297778,0.0,0.466667,47.530000,6.638889
1,0.00,0.000000,14.420000,0.327778,3.318889,0.001111,0.000000,0.953333,0.826667,0.000000,...,31.094444,0.203333,0.008889,1.820000,0.043333,0.004444,0.0,0.000000,1.604444,7.101111
2,0.00,0.000000,30.954444,5.077778,24.973333,0.024444,0.000000,3.285556,0.000000,0.254444,...,4.458889,2.487778,12.195556,3.835556,4.492222,0.100000,0.0,0.000000,4.731111,1.540000
3,0.03,0.583333,4.422222,1.601111,3.670000,0.000000,0.011111,0.813333,0.000000,0.766667,...,0.000000,0.098889,0.363333,0.002222,11.788889,0.006667,0.0,3.337778,4.556667,167.165556
4,0.00,26.812222,4.042222,0.096667,17.707778,0.000000,0.001111,0.238750,3.291111,0.167778,...,0.943333,0.405556,0.000000,0.198889,2.028889,0.051111,0.0,4.751111,0.402222,1.253333


In [126]:
y

[150.8,
 724.4,
 373.9,
 222.5,
 180.4,
 381.5,
 226.6,
 291.1,
 228.4,
 362.2,
 418.7,
 111.8,
 240.1,
 157.0,
 353.7,
 319.1,
 635.9,
 221.5,
 146.1,
 278.7,
 389.3,
 235.2,
 250.6,
 379.4,
 336.8,
 257.6,
 181.7,
 384.0,
 259.4,
 560.5,
 337.6,
 207.9,
 551.7,
 326.1,
 224.8,
 587.9,
 425.4,
 542.3,
 185.1,
 471.1,
 337.6,
 680.7,
 131.2,
 346.3,
 401.7,
 151.2,
 185.8,
 354.3,
 368.0,
 225.1,
 202.4,
 287.4,
 302.4,
 415.4,
 183.9,
 249.5,
 315.3,
 614.0,
 212.2,
 404.2,
 248.5,
 327.1,
 189.3,
 154.7,
 423.6,
 526.6,
 467.5,
 383.4,
 303.2,
 360.1,
 483.3,
 208.5,
 377.4,
 215.7,
 262.7,
 291.2,
 387.3,
 295.4,
 288.8,
 381.4,
 172.1,
 631.6,
 178.9,
 186.2,
 224.2,
 228.0,
 277.6,
 177.6,
 555.5,
 434.7,
 205.3,
 291.3,
 157.2,
 172.2,
 226.1,
 430.1,
 235.0,
 563.2,
 506.2,
 402.1,
 332.5,
 303.4,
 373.7,
 367.4,
 328.9,
 253.2,
 154.2,
 240.8,
 409.3,
 228.1,
 446.4,
 208.1,
 362.3,
 612.6,
 476.3,
 328.8,
 402.8,
 201.1,
 253.7,
 172.4,
 420.0,
 168.8,
 296.1,
 232.9,
 182.0,


In [127]:
np.asarray(x_df.iloc[:, :-1])

array([[  0.00000000e+00,   6.03333333e-01,   4.75544444e+01, ...,
          4.66666667e-01,   4.75300000e+01,   6.63888889e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.44200000e+01, ...,
          0.00000000e+00,   1.60444444e+00,   7.10111111e+00],
       [  0.00000000e+00,   0.00000000e+00,   3.09544444e+01, ...,
          0.00000000e+00,   4.73111111e+00,   1.54000000e+00],
       ..., 
       [  2.81111111e-01,   2.81422222e+01,   2.55411111e+01, ...,
          2.16222222e+00,   3.65555556e-01,   9.04833333e+01],
       [  0.00000000e+00,   0.00000000e+00,   2.94577778e+01, ...,
          6.22222222e-02,   4.32000000e+00,   8.32222222e-01],
       [  0.00000000e+00,   0.00000000e+00,   2.43888889e+01, ...,
          1.34444444e-01,   1.02344444e+01,   6.50222222e+00]])

In [234]:
model = OLS(y, np.asarray(x_df.iloc[:, :-1]))
results = model.fit()
results.rsquared
print OLS.score(model, y)

NotImplementedError: 

# Do not run things beyond this point -----

In [ ]:
end = df.shape[0]
years = [1992 + i for i in range(9)]
for crop in cropdata:
    for country in cropdata[crop]:
        if len(cropdata[crop][country]) != 9 and len(cropdata[crop][country]) != 8 and len(cropdata[crop][country]) != 0 and len(cropdata[crop][country]) != 1:
            print len(cropdata[crop]), country, crop
            

In [ ]:
print cropdata['Molasses']

In [ ]:
import copy
import time
years = [1992 + i for i in range(9)]
end = df.shape[0]
#for crop in cropdata[:1]:
N = 0
count = 0
#crop = cropdata.keys()[N]
crop = 'Ricebran Oil'
#for crop in cropdata:
countrieslist = copy.copy(df['Country'].values)
domain = df['Domain'].values[0]
element = regions_df['Element'].values[0]
unit = df['Unit'].values[0]
print len(countrieslist)
check1 = time.time()
for index, country in enumerate(countrieslist[:100]):
    if country in cancer_lands:
        for year in years[:1]:
            if year not in cropdata[crop][country]:
                count += 1
                region = country_dictionary[country]
            
                data = regiondata[region]
                check2 = time.time()
                if crop not in data:
                    df.loc[end+1] = [domain, country, element, crop, year, unit, 0]
                    end += 1
                else:
                    for yr, val in data[crop]:
                        if yr == year:
                            df.loc[end+1] = [domain, country, element, crop, year, unit, val]
                            end += 1
                            break
check3 = time.time()

#print check3-check2
print check3-check1

In [ ]:
years = [1992 + i for i in range(9)]
end = df.shape[0]
#for crop in cropdata[:1]:
N = 0
count = 0
crop = cropdata.keys()[N]
#crop = 'Ricebran Oil'
check1 = time.time()
for index, country in enumerate(df['Country'].values[:100]):
    if country in cancer_lands:
        for year in years[:1]:
            if year not in cropdata[crop][country]:
                count += 1
                region = country_dictionary[country]
                found = False
                for i, region_name in enumerate(regions_df['Country'].values):
                    if region_name == region:
                        if regions_df['Item'].values[i] == crop and regions_df['Year'].values[i] == year:
                            val = regions_df['Value'].values[i]
                            #temp = pd.DataFrame([])
                            df.loc[end+1] = [df['Domain'].values[0], country, regions_df['Element'].values[i], crop, year, regions_df['Unit'].values[i], val]
                            end += 1
                            found = True
                            break
                if not found:
                    df.loc[end+1] = [df['Domain'].values[0], country, regions_df['Element'].values[i], crop, year, regions_df['Unit'].values[i], 0]
                    end += 1
                        

check3 = time.time()
print check3 - check1

In [ ]:
df.tail(n=5)